# Agents

Explore the Agent definition with different technologies.

# Setup Notebook

## Imports

In [11]:
# Import Standard Libraries
import os
import sqlite3
import google.generativeai as genai
from dotenv import load_dotenv
from google.api_core import retry
from IPython.display import Markdown

## Load Environment Variables

In [14]:
# Load environment variables
load_dotenv('./../../.env')

# Read environment variables
google_api_key = os.getenv('GOOGLE_API_KEY')

## Configure Authentication

In [15]:
# Set the Google AI Studio API Key for genai SDK
genai.configure(api_key=google_api_key)

# Read Data

## Create Local Database

Create a local database to showcase Agent Tools usage capabilities.

In [6]:
%load_ext sql
%sql sqlite:///sample.db

In [7]:
%%sql
-- Create the 'products' table
CREATE TABLE IF NOT EXISTS products (
  	product_id INTEGER PRIMARY KEY AUTOINCREMENT,
  	product_name VARCHAR(255) NOT NULL,
  	price DECIMAL(10, 2) NOT NULL
  );

-- Create the 'staff' table
CREATE TABLE IF NOT EXISTS staff (
  	staff_id INTEGER PRIMARY KEY AUTOINCREMENT,
  	first_name VARCHAR(255) NOT NULL,
  	last_name VARCHAR(255) NOT NULL
  );

-- Create the 'orders' table
CREATE TABLE IF NOT EXISTS orders (
  	order_id INTEGER PRIMARY KEY AUTOINCREMENT,
  	customer_name VARCHAR(255) NOT NULL,
  	staff_id INTEGER NOT NULL,
  	product_id INTEGER NOT NULL,
  	FOREIGN KEY (staff_id) REFERENCES staff (staff_id),
  	FOREIGN KEY (product_id) REFERENCES products (product_id)
  );

-- Insert data into the 'products' table
INSERT INTO products (product_name, price) VALUES
  	('Laptop', 799.99),
  	('Keyboard', 129.99),
  	('Mouse', 29.99);

-- Insert data into the 'staff' table
INSERT INTO staff (first_name, last_name) VALUES
  	('Alice', 'Smith'),
  	('Bob', 'Johnson'),
  	('Charlie', 'Williams');

-- Insert data into the 'orders' table
INSERT INTO orders (customer_name, staff_id, product_id) VALUES
  	('David Lee', 1, 1),
  	('Emily Chen', 2, 2),
  	('Frank Brown', 1, 3);

 * sqlite:///sample.db
Done.
Done.
Done.
Done.
Done.
Done.


[]

# Tools

## Data Stores

### SQLite Data Stores with Gemini

In [9]:
# Connect to DB
db_file = "sample.db"
db_conn = sqlite3.connect(db_file)

In [10]:
# DB helper functions
def list_tables() -> list[str]:
    """Retrieve the names of all tables in the database."""
    # Include print logging statements so you can see when functions are being called.
    print(' - DB CALL: list_tables')

    cursor = db_conn.cursor()

    # Fetch the table names.
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

    tables = cursor.fetchall()
    return [t[0] for t in tables]

def describe_table(table_name: str) -> list[tuple[str, str]]:
    """Look up the table schema.

    Returns:
      List of columns, where each entry is a tuple of (column, type).
    """
    print(' - DB CALL: describe_table')

    cursor = db_conn.cursor()

    cursor.execute(f"PRAGMA table_info({table_name});")

    schema = cursor.fetchall()
    # [column index, column name, column type, ...]
    return [(col[1], col[2]) for col in schema]

def execute_query(sql: str) -> list[list[str]]:
    """Execute a SELECT statement, returning the results."""
    print(' - DB CALL: execute_query')

    cursor = db_conn.cursor()

    cursor.execute(sql)
    return cursor.fetchall()

In [16]:
# Define the Agent tools
db_tools = [list_tables, describe_table, execute_query]

# Define system prompt
instruction = """You are a helpful chatbot that can interact with an SQL database for a computer
store. You will take the users questions and turn them into SQL queries using the tools
available. Once you have the information you need, you will answer the user's question using
the data returned. Use list_tables to see what tables are present, describe_table to understand
the schema, and execute_query to issue an SQL SELECT query."""

# Define Agent LLM's model
model = genai.GenerativeModel(
    "models/gemini-1.5-flash-latest", tools=db_tools, system_instruction=instruction
)

# Define a retry policy
retry_policy = {"retry": retry.Retry(predicate=retry.if_transient_error)}

# Start a chat
chat = model.start_chat(enable_automatic_function_calling=True)

In [17]:
# Inference
resp = chat.send_message("What is the cheapest product?", request_options=retry_policy)
Markdown(resp.text)

 - DB CALL: list_tables
 - DB CALL: describe_table
 - DB CALL: execute_query


The cheapest product is a Mouse. 


In [18]:
response = chat.send_message('Which salesperson sold the cheapest product?', request_options=retry_policy)
Markdown(response.text)

 - DB CALL: execute_query


The salesperson with id 1 sold the cheapest product. 
